In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
NB_DAYS = 250
NB_YEARS = 30

DF_QUANTILE = 1. - (1. / (NB_YEARS * NB_DAYS))

In [3]:
def compute_default_fund(nu, im_quantle, df_quantile):
    dir_sim_ = './simulations'
    dir_tr_input_ = './transformed_input'
    dir_res_ = './results'
    
    udl_distrib = pd.read_csv(os.path.join(dir_sim_,'nu_eq_%i.csv' % nu), header=0, index_col=0)
    members_pos = pd.read_csv(os.path.join(dir_tr_input_, 'positions.csv'), header=0, index_col=0)

    loss_and_profit = members_pos.dot(udl_distrib)
    
    im = pd.read_csv(os.path.join(dir_res_, 'IM_nu_eq_%i.csv' % nu), header=0, index_col=0)
    
    loim_q1 = loss_and_profit.subtract(im[str(im_quantle)], axis=0).quantile(df_quantile, axis=1)
    loim_q2 = (-loss_and_profit).subtract(im[str(im_quantle)], axis=0).quantile(df_quantile, axis=1)

    tmp = pd.DataFrame([loim_q1, loim_q2]).max()
    tmp = tmp.sort_values()[::-1]
    
    cover2 = np.maximum(tmp.iloc[0], tmp.iloc[1] + tmp.iloc[2]) * np.sqrt(5./3.)
    return cover2    

In [4]:
for im in (99., 99.7):    
    for nu in (2, 6, 50):
        print "IM = %s, nu = %s, Cover2 = %s" % (im, nu, compute_default_fund(nu, im, DF_QUANTILE))

IM = 99.0, nu = 2, Cover2 = 615558632.151
IM = 99.0, nu = 6, Cover2 = 672189852.67
IM = 99.0, nu = 50, Cover2 = 626709845.651
IM = 99.7, nu = 2, Cover2 = 496292831.931
IM = 99.7, nu = 6, Cover2 = 547736179.105
IM = 99.7, nu = 50, Cover2 = 500048647.076
